In [ ]:
#  Copyright (c) 2017-2025 KAPPA
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# This file is part of Orchid and related technologies.
#

# Tutorial DOM walk-through

This notebook is a live tutorial on illustrating how to navigate through an Orchid* project.

(*Orchid is a mark of KAPPA)

## 0.5 Import packages

The only import needed for the Python API is `orchid` itself.

In [ ]:
import orchid

Import other packages to meet specific needs

In [ ]:
import uuid  # Used to construct an object ID from its string representation

## 1.0 Load the .ifrac project

The following code simply captures the configured location of the Orchid training data. It is not needed to
use the Orchid Python API itself, but it is used in this example to load a well-known `.ifrac` file.

In [ ]:
orchid_training_data_path = orchid.training_data_path()

In [ ]:
project = orchid.load_project(str(orchid_training_data_path.joinpath(
    'frankNstein_Bakken_UTM13_FEET.ifrac')))

Our project is now loaded in memory. An Orchid project has many collections of other items. For example, a
project has a collection of wells, and a well has a collection of stages.

Each of these objects, for example, each well and each stage, is identified by a unique identifier (an
instance of `uuid.UUID`). However, these identifiers, in order to be unique, are **not** easily remembered by
people. Further, Orchid **does not** require that alternatives, like a well name or display name, be unique.
To allow for convenient searching, project objects like wells and stages are kept in a
`SearchableProjectObjects` collection. This class provides methods for searching for more specific instances:

- `find_by_object_id()` - Returns the matching object or `None` if no such object exists
- `find_by_name()` - Returns an **iterator** of matching objects (since more than one may match).
- `find_by_display_name()` - Returns an **iterator** of matching objects.

It provides methods returning all valid values of these keys:

- `all_object_ids()`
- `all_names()`
- `all_display_names()`

Since `find_by_object_id()`, `find_by_name()` and `find_by_display_name()` do not exhaust the criteria you
might want to use to find objects of interest, we have included a more generic method, `find()`, that takes a
predicate (a callable) and returns an iterator over all objects for which the predicate returns `True`.

Finally, we provide the general ability to iterate over all project objects using two mechanisms. The
older mechanism is the `all_objects()` method that returns an iterator over all objects in the collection.

Beginning with version 2021.3.318 of the Orchid Python API, we have made `SearchableProjectObjects` an
iterator. This change allows one to use the very familiar Python idiom of:

```
for well in projects.wells():
    // processing for a single well
```

Additionally, one can use `SearchableProjectObjects` in list, dictionary and generator expressions. For
example, to create a dictionary of wells keyed by the well object ID (since **only** object ID is guaranteed
to be unique) one would write:

```
wells_by_id = {well.object_id: well for well in project.wells()}
```

Using `SearchableProjectObjects` as an iterator is very general, is easily understood, and is most useful when
exploring a project interactively at a REPL. However, we have seen that using this general technique
easily leads to duplicated code to search through the collection of project objects for the specific object
that you actually need for your application. Consequently, we encourage you to use the query methods of
`SearchableProjectObjects`, including the general `find(...)` methods, first. If those do not meet your needs,
please let us know by creating [an issue](https://github.com/Reveal-Energy-Services/orchid-python-api/issues).

## 2.0 Query well "keys"

Particularly during exploration of a project, you may not know the specific object in which you are
interested, but you know something about its name or its display name. The Orchid Python API provides you
with the `all_names()` and `all_display_names()` to iterate over those names.

In [ ]:
all_well_names = list(project.wells().all_names())
print(f"all_well_names = {all_well_names}")

In [ ]:
all_well_display_names = list(project.wells().all_display_names())
print(f"all_well_display_names = {all_well_display_names}")

### 2.1 Query all object ids

For completeness, we provide the `all_object_ids()` to list all the object IDs.

In [ ]:
all_well_object_ids = list(project.wells().all_object_ids())
print(f"all_well_object_ids = {all_well_object_ids}")

## 3.0 Find well by "key"

The method, `find_by_name()`, returns an iterable over wells.

In [ ]:
wells_of_interest_by_name = list(project.wells().find_by_name('Demo_1H'))
[(well.name, well.display_name, well.object_id) for well in wells_of_interest_by_name]

Similarly, the method, `find_by_display_name()`, returns an iterable over wells

In [ ]:
wells_of_interest_by_display_name = list(project.wells().find_by_display_name('Demo_2H'))
[(well.name, well.display_name, well.object_id) for well in wells_of_interest_by_display_name]

Because `find_by_name()` and `find_by_display_name()` returns an **iterator**, one typically must handle this
method returning

- An empty iterator
- An iterator with more than 1 item
- An iterator with a single item

For example,

In [ ]:
well_name_of_interest = 'Demo_3H'
wells_of_interest_by_name = list(project.wells().find_by_name(well_name_of_interest))
if len(wells_of_interest_by_name) == 0:
    print(f'No well in project with name, {well_name_of_interest}')
elif len(wells_of_interest_by_name) > 1:
    print(f'Found multiple wells ({len(wells_of_interest_by_name)}) in project with name,'
          f' {well_name_of_interest}')
else:
    print(f'Found single well in project with name, {well_name_of_interest}')

Another way to handle multiple wells found by `find_by_name()` is to use `assert` statements. This way is
suitable if any number of matches other than one is an error.

In [ ]:
assert len(wells_of_interest_by_name) == 1, (f'Expected one well with name, {well_name_of_interest},'
                                             f' but found {len(wells_of_interest_by_display_name)}')
well_of_interest = wells_of_interest_by_name[0]
well_of_interest.name, well_of_interest.display_name, well_of_interest.object_id

However, `find_by_object_id()` method returns either a well with the specified object ID or None.

If a well with this object ID exists:

In [ ]:
object_id = '9fe727b0-5fd1-4240-b475-51c1363edb0d'
well_of_interest_by_object_id = project.wells().find_by_object_id(uuid.UUID(object_id))
((well_of_interest_by_object_id.name,
  well_of_interest_by_object_id.display_name,
  well_of_interest_by_object_id.object_id) if well_of_interest_by_object_id is not None
 else "No such object")

But if no well with this object ID exists:

In [ ]:
object_id = '9fe727b0-5fd1-4240-b475-51c1363edb0e'
well_of_interest_by_object_id = project.wells().find_by_object_id(uuid.UUID(object_id))
((well_of_interest_by_object_id.name,
  well_of_interest_by_object_id.display_name,
  well_of_interest_by_object_id.object_id) if well_of_interest_by_object_id is not None
 else "No such object")

## 4.0 The `find()` method supports more generic queries

The `find()` method returns an iterable over the wells for which the specified predicate is `True`

In [ ]:
wells_of_interest = list(project.wells().find(lambda well: well.name == 'Demo_3H' or well.display_name == 'Demo_4H'))
for well_of_interest in wells_of_interest:
    print(f'well_of_interest=Well(name={well_of_interest.name},',
          f'display_name={well_of_interest.display_name}, '
          f'object_id={well_of_interest.object_id})')

## 5.0 Finally, we offer two ways to iterator over all the objects

The method, `all_objects()`, returns an iterable over **all** wells in the project

In [ ]:
wells_of_interest = list(project.wells().all_objects())
[(well.name, well.display_name, well.object_id) for well in wells_of_interest]

The `SearchableProjectObjects` class is an iterator and supports typical Python iterator idioms like:

In [ ]:
[(well.name, well.display_name, well.object_id) for well in project.wells()]

All the project top-level objects provide a similar interface:

- `project.data_frames()`
- `project.monitors()`
- `project.time_series()`
- `project.wells()`

Stages have the same interface; however, stages also have two additional methods:

- `stage.find_by_display_stage_number()`
- `stage.find_by_display_name_with_well()`

## 6.0 Additional `SearchableStages` methods

### 6.1 Find stages with a specific display stage number

The method, `find_by_display_stage_number()`, returns either the single stage with the display
stage number or it returns `None`.

Begin by searching for the well of interest.

In [ ]:
well_display_name_of_interest = 'Demo_1H'
wells_of_interest_by_display_name = [
    well for well in project.wells().find_by_display_name(well_display_name_of_interest)
]

assert len(wells_of_interest_by_display_name) == 1,\
    f'Expected one well with display_name, {well_display_name_of_interest},' \
    f' but found {len(wells_of_interest_by_display_name)}'
well_of_interest = wells_of_interest_by_display_name[0]
well_of_interest.name, well_of_interest.display_name, well_of_interest.object_id

Now search for a stage with a specified display stage number.

In [ ]:
stage_display_number_of_interest = 7
stage_of_interest = well_of_interest.stages().find_by_display_stage_number(stage_display_number_of_interest)
if stage_of_interest is not None:
    print((stage_of_interest.name, stage_of_interest.display_name, stage_of_interest.display_stage_number))
else:
    print(f'No stage with display stage number, {stage_display_number_of_interest}.')

If you search for a stage by display stage number that does **not** exist:

In [ ]:
stage_display_number_of_interest = 9999
stage_of_interest = well_of_interest.stages().find_by_display_stage_number(stage_display_number_of_interest)
if stage_of_interest is not None:
    print((stage_of_interest.name, stage_of_interest.display_name, stage_of_interest.display_stage_number))
else:
    print(f'No stage with display stage number, {stage_display_number_of_interest}.')

### 6.2 Find stages with a specific display name with well

The method, `find_by_display_name_with_well()`, returns:

- An empty iterator if no stage with the specified display name with well exists
- An iterator with a single item if exactly one stage with the specified display name with well exists
- An iterator with multiple items if many stages with the specified display name with well exists

For example, begin by searching for a well of interest.

In [ ]:
well_name_of_interest = 'Demo_2H'
wells_of_interest_by_name = list(project.wells().find_by_name(well_name_of_interest))
assert len(wells_of_interest_by_name) == 1, (f'Expected one well with name, {well_name_of_interest},'
                                             f' but found {len(wells_of_interest_by_display_name)}')
well_of_interest = wells_of_interest_by_name[0]
well_of_interest.name, well_of_interest.display_name, well_of_interest.object_id

In [ ]:
stage_display_name_with_well_of_interest = 'Demo_2H-Stage-14'
stages_of_interest = [stage for stage
                      in well_of_interest.stages().find_by_display_name_with_well
                      (stage_display_name_with_well_of_interest)]
if len(stages_of_interest) == 0:
    print(f'No stage in well, {well_of_interest.display_name}, in project with name, '
          f'{stage_display_name_with_well_of_interest}')
elif len(stages_of_interest) > 1:
    print(f'Found multiple stages ({len(stages_of_interest)}) in well,'
          f' {well_of_interest.display_name}, with display name with well,'
          f' {stage_display_name_with_well_of_interest}')
else:
    print(f'Found single stage in well, {well_of_interest.display_name},'
          f' with display name withe well, {stage_display_name_with_well_of_interest}')

Another way to handle multiple stages found by `find_by_display_name_with_well()` is to use `assert`
statements. This way is suitable if any number of matches other than one is an error.

In [ ]:
assert len(stages_of_interest) == 1, \
    (f'Expected one well with name, ' f'{stage_display_name_with_well_of_interest},'
     f' but found {len(stages_of_interest)}')
stage_of_interest = stages_of_interest[0]
stage_of_interest.name, stage_of_interest.display_name, stage_of_interest.display_name_with_well